In [23]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
import re
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [24]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [25]:
english_stops = set(stopwords.words('english'))

In [26]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\Prachi Shedge\AppData\Local\Temp\ipykernel_9508\4067306172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [27]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
5642     [i, kind, like, bam, margera, i, curious, but,...
32850    [be, warned, of, possible, spoilers, a, friend...
40764    [having, spent, money, caring, terminally, ill...
35329    [what, earth, my, family, i, waisted, hours, l...
9753     [ok, who, brought, cheese, i, love, during, ru...
                               ...                        
41183    [i, finishing, watching, goliath, awaits, i, o...
9848     [this, movie, wife, i, ever, walked, totally, ...
14908    [in, summer, also, boasted, repugnant, stinker...
5868     [this, case, script, plays, audience, manner, ...
13852    [a, confusing, senseless, script, plot, holes,...
Name: review, Length: 40000, dtype: object 

42905    [drab, morality, tale, high, school, kid, pret...
43878    [william, wyler, directed, adaptation, moss, h...
24668    [the, kid, at, years, old, russel, duritz, lif...
10667    [i, think, film, one, funniest, films, ever, s...
9275     [kalifornia, story, writer, girlfriend, photog...
 

In [28]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [29]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    1   147     6 ...     0     0     0]
 [ 2135  2571   275 ... 46063 28073   325]
 [ 1502   915   197 ...  2143   213  1661]
 ...
 [   49  1374    22 ...    81  2973    11]
 [    9   328   133 ...    83   138    58]
 [   39  1342  4852 ...     0     0     0]] 

Encoded X Test
 [[ 6761  3773   690 ...    20     2  1384]
 [  909 17636   429 ...  4249  5823  9404]
 [    2   431   290 ... 13470 41282    97]
 ...
 [ 1001    37   759 ...     0     0     0]
 [ 1248 13509 10107 ... 11509    60    99]
 [  795  6823    99 ...     0     0     0]] 

Maximum review length:  130


In [30]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

C:\Users\Prachi Shedge\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [32]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.5322 - loss: 0.6882
Epoch 1: accuracy improved from -inf to 0.53175, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 52s 151ms/step - accuracy: 0.5322 - loss: 0.6882
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.5618 - loss: 0.6862
Epoch 2: accuracy improved from 0.53175 to 0.56175, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 150ms/step - accuracy: 0.5618 - loss: 0.6862
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.6289 - loss: 0.6457
Epoch 3: accuracy improved from 0.56175 to 0.65715, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 149ms/step - accuracy: 0.6290 - loss: 0.6457
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.7155 - loss: 0.5744
Epoch 4: accuracy did not improve from 0.65715
313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 150ms/step - accuracy: 0.7152 - loss: 0.5746
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.6344 - loss: 0.6388
Epoch 5: accuracy did not improve from 0.65715
313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 150ms/step - accuracy: 0.6344 - loss: 0.6388


In [33]:
y_pred = model.predict(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step
Correct Prediction: 0
Wrong Prediction: 10000
Accuracy: 0.0


In [34]:
loaded_model = load_model('models/LSTM.h5')

In [35]:
review = str(input('Movie Review: '))

In [36]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Enter
Filtered:  ['enter']


In [37]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[2229    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [38]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
[[0.89425856]]


In [39]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
